<a href="https://colab.research.google.com/github/songqsh/18ma573pub/blob/master/src/implied_vol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implied volatility

## Abstract 


- We will introduce implied volatility and its computations

## Problem

We have seen that BS call price is 5.94 with the following parameters:

In [0]:
vol_ratio = .2; spot_price = 100.; drift_ratio = .0475; strike = 110.; maturity = 1.



In the market, usually volatility (vol = 0.2 in the above) is not available, but market price (call = 5.94) for the option is available. One wants to find a volatility such that associated BSM formula is equal to (fit) the market price. This is called implied volatility. 

Next, we are goint to find implied volatility given the following parameters:

spot_price = 100.; drift_ratio = .0475; strike = 110.; maturity = 1.; market_call_price = 5.94

## Anal

the implied volatility of an option contract is that value of the volatility of the underlying instrument which, when input in an option pricing model (such as Black–Scholes) will return a theoretical value equal to the current market price of the option. 

Consider a derivative or a call option particularly, we know parameter sets of 
- spot price.; drift_ratio (interest rate); strike.; maturity

For a BSM theoretical price, the only missing parameter is the volatility $\sigma$. Let's denote a function $f$ be
$$f: \sigma \mapsto BSM(\sigma, others).$$
Suppose we denote by $p$ for the market price of the call option, then the implied volatility is
$\hat \sigma$ satisfying
$$f(\hat \sigma) = p$$
or 
$$\hat \sigma = f^{-1}(p),$$
if the inverse of $f$ is well defined at $p$.

The next lemma tells us, we can search for the minimizer of the error function
$$\sigma \mapsto |f(\sigma) - p|,$$
which is available by "scipy.optimize.fmin". 

__Lemma__

Supose $f$ is a function satisfying
- $f: [0, \infty) \mapsto (f_{min}, f_{max})$
- $f$ is continuous
- $f$ is strictly increasing
then, for any $p\in  (f_{min}, f_{max})$, 
- there exists unique $\hat \sigma$, s.t. $f(\hat \sigma) = p$;
- Moreover, 
$$\hat \sigma = \arg\min_{\sigma\in (0,\infty)} | f(\sigma) - p|.$$

## Code

In [2]:
'''======
run once, then comment it off, and run again
========'''
!git clone https://github.com/songqsh/18ma573pub.git 
pass

Cloning into '18ma573pub'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 331 (delta 20), reused 2 (delta 0), pack-reused 294
Receiving objects: 100% (331/331), 565.75 KiB | 12.86 MiB/s, done.
Resolving deltas: 100% (175/175), done.


Change working directory

In [3]:
cd 18ma573pub/src/

/content/18ma573pub/src


In [0]:
'''==========
here we import our own python modules
==================='''
import european_options_class as eoc
import sde_class as sde

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so

In [6]:
'''===============
Test bsm_price
================='''
gbm1 = sde.Gbm(init_state = 100.,
                 drift_ratio = .0475,
                 vol_ratio = .2)
option1 = eoc.EuropeanOption(
    otype = 1, # 1: 'call'
              # -1: 'put'
    strike = 110.,                
    maturity = 1.
)
print('>>>>>>>call value is ' + str(gbm1.bsm_price(option1)))
option2 = eoc.EuropeanOption(otype=-1)
print('>>>>>>>>put value is ' + str(gbm1.bsm_price(option2)))


>>>>>>>call value is 5.943273183452838
>>>>>>>>put value is 10.84042522804176


In the market, usually volatility (vol = 0.2 in the above) is not available, but market price (call = 5.94) for the option is available. One wants find a volatility such that associated BSM formula is equal to (fit) the market price. 

In [0]:
'''==========
define a method to seek for an implied volatility
============'''

def implied_volatility(spot_price, otype, maturity, strike, 
                       interest_rate, market_option_price):
    option = eoc.EuropeanOption(otype, strike, maturity)
    init_vol = .1 #initial guess
    gbm = sde.Gbm(spot_price, interest_rate, init_vol)
    
    def error_function(vol):
        gbm.vol_ratio = vol
        return np.abs(market_option_price - gbm.bsm_price(option))
    
    return so.fmin(error_function, 0.1, disp = False)[0]

In [8]:
'''============
test the implied_vol by reversing bsm_formula example in the above
=============='''
print('>>>>>>>>implied volatility is')
implied_volatility(spot_price=100, otype=1, maturity=1, 
                   strike=110, interest_rate=.0475, market_option_price=5.94)

>>>>>>>>implied volatility is


0.19992187500000036